# GLACIS Offline Attestation Demo (Development Mode)

**What is GLACIS?** A library that creates tamper-proof receipts for AI operations.
Prove what your AI did, what data it saw - without sensitive data leaving your environment.

## What You'll Learn
1. **Offline Attestation** - Self-signed attestations, no GLACIS API key needed
2. **Control Pipeline** - Staged input/output controls: PII/PHI detection, jailbreak detection, word filtering
3. **Control Plane Attestation** - Cryptographically bound control results

## Offline Mode: Perfect for Development
- No API key required
- Receipts are self-signed locally (Ed25519)
- Full control pipeline works identically to online mode
- `witness_status` = `UNVERIFIED` (not witnessed by server)

**Ready for production?** See the [Online Attestation Demo](online_attestation_demo.ipynb) for server-witnessed attestations with Merkle proofs.

---
## Setup

```bash
pip install glacis[openai,controls]   # OpenAI + PII detection + jailbreak detection
```

For full NER-based detection, also install the spaCy model:
```bash
python -m spacy download en_core_web_md
```

In [ ]:
import os
from pathlib import Path

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    env_path = Path("../.env") if Path("../.env").exists() else Path(".env")
    load_dotenv(env_path)
    print(f"Loaded environment from: {env_path}")
except ImportError:
    print("python-dotenv not installed, using system environment variables")

# Check for OpenAI API key
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    print("WARNING: No OPENAI_API_KEY found. Set it to run the control stack demo.")
else:
    print("OpenAI API key loaded")

# Generate a 32-byte signing seed for Ed25519 (offline mode)
# In production, persist this securely to maintain the same identity
SIGNING_SEED = os.urandom(32)
print(f"Generated signing seed: {SIGNING_SEED.hex()[:16]}...")

---
## 1. Basic Offline Attestation

Create a self-signed attestation. No API key needed - perfect for local development.

In [ ]:
from glacis import Glacis

# Create an offline Glacis client
glacis = Glacis(mode="offline", signing_seed=SIGNING_SEED)
print(f"Glacis client mode: {glacis.mode}")

In [ ]:
# Simulate an AI interaction
prompt = "What is the capital of France?"
response = "The capital of France is Paris."

# Create an attestation
receipt = glacis.attest(
    service_id="demo-app",
    operation_type="inference",
    input={"prompt": prompt},
    output={"response": response},
    metadata={"model": "gpt-4", "temperature": 0.7},
)

print("Attestation created!")
print(f"  Receipt ID:      {receipt.id}")
print(f"  Evidence Hash:   {receipt.evidence_hash[:32]}...")
print(f"  Public Key:      {receipt.public_key[:32]}...")
print(f"  Signature:       {receipt.signature[:32]}...")
print(f"  Witness Status:  {receipt.witness_status}")

In [ ]:
# Verify the receipt (signature only in offline mode)
result = glacis.verify(receipt)

print("Verification result:")
print(f"  Signature Valid: {result.signature_valid}")
print(f"  Overall Valid:   {result.valid}")
print(f"  Witness Status:  {result.witness_status}")

---
## 2. Control Pipeline: PII Detection + Jailbreak Detection

The control pipeline runs in two stages (configured in `glacis.yaml` v1.3):
- **Input stage**: Controls run before the LLM call (PII scanning, jailbreak detection)
- **Output stage**: Controls run after the LLM call (PII redaction, content filtering)

Each stage has two layers:
- **Redacting layer** (sequential): PII, word filter — text chains through `modified_text`
- **Scanning layer** (parallel): Jailbreak detection — read-only

In [ ]:
if OPENAI_API_KEY:
    from glacis.integrations.openai import attested_openai, get_last_receipt

    # Create an attested OpenAI client with controls enabled (via glacis.yaml)
    client = attested_openai(
        openai_api_key=OPENAI_API_KEY,
        offline=True,
        signing_seed=SIGNING_SEED,
        debug=True,  # Show control execution
    )
    print("Attested OpenAI client ready (offline mode with controls)!")
else:
    print("Skipped - no OPENAI_API_KEY")

In [ ]:
if OPENAI_API_KEY:
    from glacis.integrations.openai import get_evidence

    # Clinical query with PHI (will be detected and reported, but sent unchanged)
    system_prompt = """You are a clinical decision support assistant for healthcare providers.
Your role is to help with medication interaction checks and treatment recommendations.
Focus on the clinical question and provide evidence-based guidance."""

    clinical_query = """
I need a medication interaction check for my patient.

Patient: Margaret Chen
DOB: 11/22/1956
MRN: MRN-4829173
Contact: margaret.chen@gmail.com, (415) 555-8234

Current Medications:
- Warfarin 5mg daily (target INR 2-3 for afib)
- Metoprolol 50mg twice daily
- Lisinopril 20mg daily
- Atorvastatin 40mg at bedtime

The patient reports worsening knee pain from osteoarthritis. She's asking about
starting over-the-counter ibuprofen (Advil) 400mg three times daily.

What are the key drug interactions I should consider?
"""

    print("=" * 70)
    print("ORIGINAL QUERY (contains PHI)")
    print("=" * 70)
    print(clinical_query)

    print("\n" + "=" * 70)
    print("CALLING OPENAI (with control pipeline)...")
    print("=" * 70 + "\n")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": clinical_query}
        ],
        max_tokens=300,
    )

    # Show what controls detected
    receipt = get_last_receipt()
    evidence = get_evidence(receipt.id)
    if evidence:
        cp = evidence.get('control_plane_results')
        if cp:
            print("=" * 70)
            print("CONTROL PIPELINE RESULTS")
            print("=" * 70)
            det = cp.get('determination', {})
            print(f"  Action: {det.get('action')}")
            for ctrl in cp.get('controls', []):
                stage = ctrl.get('stage', 'input')
                score_str = f", score={ctrl['score']:.2f}" if ctrl.get('score') is not None else ""
                print(f"  [{stage}] {ctrl.get('id')}: {ctrl.get('status')}{score_str}")

    print("\n" + "=" * 70)
    print("CLINICAL RESPONSE")
    print("=" * 70)
    print(response.choices[0].message.content)
else:
    print("Skipped - no OPENAI_API_KEY")

---
## 3. Control Plane Attestation Report

The attestation receipt includes control plane results showing what controls executed across input and output stages.

In [ ]:
if OPENAI_API_KEY:
    # Get the receipt from the last call
    receipt = get_last_receipt()

    print("=" * 70)
    print("ATTESTATION RECEIPT (OFFLINE - SELF-SIGNED)")
    print("=" * 70)
    print(f"  Receipt ID:     {receipt.id}")
    print(f"  Timestamp:      {receipt.timestamp}")
    print(f"  Witness Status: {receipt.witness_status}")
    if receipt.cpr_hash:
        print(f"  CPR Hash:       {receipt.cpr_hash[:32]}...")

    if receipt.control_plane_results:
        cp = receipt.control_plane_results
        controls = cp.get("controls", [])
        print(f"\n  Controls Executed: {len(controls)}")
        for ctrl in controls:
            stage = ctrl.get('stage', 'input')
            score_str = f", score={ctrl['score']:.2f}" if ctrl.get('score') is not None else ""
            print(f"    - [{stage}] {ctrl['id']}: {ctrl['status']} ({ctrl['latency_ms']}ms{score_str})")

        det = cp.get("determination", {})
        print(f"\n  Determination: {det.get('action')}")
else:
    print("Skipped - no OPENAI_API_KEY")

In [ ]:
# Cleanup
glacis.close()
print("Done!")

---
## Why Upgrade to GLACIS Online?

Offline mode is great for **development and testing**, but production systems need **third-party verifiability**.

| Feature | Offline | Online |
|---------|:-------:|:------:|
| Self-signed attestations | Yes | Yes |
| Control pipeline (PII + jailbreak + word filter) | Yes | Yes |
| Local evidence storage | Yes | Yes |
| **Server-witnessed receipts** | - | Yes |
| **Merkle tree inclusion proofs** | - | Yes |
| **Shareable badge URL** | - | Yes |
| **Query transparency log** | - | Yes |
| **Compliance export** | - | Yes |
| **Tamper-evident audit trail** | - | Yes |

### What Online Mode Gives You

1. **Third-Party Verification**: Anyone can verify your attestations via badge URL - no access to your systems required
2. **Merkle Proofs**: Cryptographic proof that your attestation is included in the transparency log
3. **Compliance Export**: Query past attestations for audit reports and regulatory compliance
4. **Tamper Evidence**: If anyone modifies the log, the Merkle root changes - instantly detectable

### Get Started

**Get your API key at [https://glacis.io](https://glacis.io)**

Then check out the [Online Attestation Demo](online_attestation_demo.ipynb) for the full audit workflow.